In [75]:
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer,HashingVectorizer
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from sklearn.metrics import accuracy_score
from sklearn.ensemble import GradientBoostingClassifier,AdaBoostClassifier
from sklearn.linear_model import SGDClassifier
import xgboost
from sklearn.model_selection import cross_val_score

In [76]:
ADA=AdaBoostClassifier()

In [77]:
XGC=xgboost.XGBClassifier()

In [78]:
df=pd.read_csv("dataset.csv")
df

,inline-comment-id,# Comment,Question,Final Label
0,84326dd1_566c7146,1,is this what they intended? don't they really ...,request for confirmation
1,84326dd1_566c7146,2,is this what they intended? don't they really ...,surprise
2,99d1f8e4_92b31cea,3,Don't we need to increment 'i' in the else cas...,suggestion
3,193d089f_f5fac752,4,i can't see anywhere where this is set to fals...,suggestion
4,50c2f81e_ac4fd6fc,5,are you sure you want to include this source f...,criticism
...,...,...,...,...
494,cafaac81_2fec65e1,495,does this work for glibc? they have a FORTIFY ...,request for confirmation
495,bf1dbc5e_1aa99db1,496,"extern ""C"" but using bool without <stdbool.h> ...",surprise
496,bf1dbc5e_1aa99db1,497,"extern ""C"" but using bool without <stdbool.h> ...",suggestion
497,9f439833_6e39afeb,498,Did you hit build issues here or is this just ...,request for information


In [79]:
df["Final Label"].value_counts()
df["Final Label"]=df["Final Label"].replace({"suggestion":1,
           "hypothetical scenario":4,
           "rhetorical question":2,
           })

In [80]:
df.loc[df['Final Label'].str.contains('request',na=False), 'Final Label'] =3

In [81]:
df["Final Label"]=df["Final Label"].apply(lambda x: 5 if type(x)==str else x)

In [83]:
corpus =[]
for i in range(0, len(df)):
    text = re.sub('[^a-zA-Z]', ' ', df['Question'][i]).strip().lower()
    text=text.split()
    sb = SnowballStemmer(language="english")
    all_stopwords = stopwords.words('english')
    text = [sb.stem(word) for word in text if not word in set(all_stopwords)]
    text = ' '.join(text)
    corpus.append(text)

In [84]:
df

,inline-comment-id,# Comment,Question,Final Label
0,84326dd1_566c7146,1,is this what they intended? don't they really ...,3
1,84326dd1_566c7146,2,is this what they intended? don't they really ...,5
2,99d1f8e4_92b31cea,3,Don't we need to increment 'i' in the else cas...,1
3,193d089f_f5fac752,4,i can't see anywhere where this is set to fals...,1
4,50c2f81e_ac4fd6fc,5,are you sure you want to include this source f...,5
...,...,...,...,...
494,cafaac81_2fec65e1,495,does this work for glibc? they have a FORTIFY ...,3
495,bf1dbc5e_1aa99db1,496,"extern ""C"" but using bool without <stdbool.h> ...",5
496,bf1dbc5e_1aa99db1,497,"extern ""C"" but using bool without <stdbool.h> ...",1
497,9f439833_6e39afeb,498,Did you hit build issues here or is this just ...,3


In [85]:
new=[]
import random
for i in range(len(df)):
    new+=[(all_stopwords[random.randint(0,170)])]

In [86]:
new=pd.DataFrame(new)

In [87]:
df=df.join(new)


In [88]:
df

,inline-comment-id,# Comment,Question,Final Label,0
0,84326dd1_566c7146,1,is this what they intended? don't they really ...,3,doesn
1,84326dd1_566c7146,2,is this what they intended? don't they really ...,5,i
2,99d1f8e4_92b31cea,3,Don't we need to increment 'i' in the else cas...,1,don
3,193d089f_f5fac752,4,i can't see anywhere where this is set to fals...,1,which
4,50c2f81e_ac4fd6fc,5,are you sure you want to include this source f...,5,haven't
...,...,...,...,...,...
494,cafaac81_2fec65e1,495,does this work for glibc? they have a FORTIFY ...,3,she's
495,bf1dbc5e_1aa99db1,496,"extern ""C"" but using bool without <stdbool.h> ...",5,after
496,bf1dbc5e_1aa99db1,497,"extern ""C"" but using bool without <stdbool.h> ...",1,couldn
497,9f439833_6e39afeb,498,Did you hit build issues here or is this just ...,3,yourselves


In [89]:
Y=df.iloc[:,-2]
Y

0      3
1      5
2      1
3      1
4      5
      ..
494    3
495    5
496    1
497    3
498    3
Name: Final Label, Length: 499, dtype: int64

In [90]:
from sklearn.model_selection import train_test_split

In [91]:
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', ngram_range=(1, 2), stop_words='english')
X = tfidf.fit_transform(corpus).toarray()
y = df.iloc[:, -1].values

In [92]:
xt,xs,yt, ys = train_test_split(X, Y, test_size = 0.3, random_state = 0)

In [93]:
from sklearn.svm import SVC
classifier = SVC(kernel = 'sigmoid', random_state = 0)
classifier.fit(xt, yt)
y_pred = classifier.predict(xs)

In [94]:
from sklearn.metrics import accuracy_score
accuracy_score(ys, y_pred)

0.58

In [95]:
# gridsc=GridSearchCV(corpus, parameters, n_jobs=-1)
# gridsc.fit(Xtdi,Y)

In [96]:
from sklearn.pipeline import Pipeline
text_clf = Pipeline([('vect', TfidfVectorizer()),
                      ('clf',SGDClassifier(loss='modified_huber', penalty='elasticnet',
                                            alpha=1e-3, random_state=42))])
text_clf = text_clf.fit(corpus,Y)
pred=text_clf.predict(corpus[350:])
accuracy_score(pred,Y[350:] )

0.9261744966442953

In [97]:
from sklearn.pipeline import Pipeline
text_clf = Pipeline([('vect', TfidfVectorizer()),
                      ('clf',XGC)])
text_clf = text_clf.fit(corpus,Y)
pred=text_clf.predict(corpus[350:])
accuracy_score(pred,Y[350:] )

C:\Anaconda_All\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[15:58:24] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


0.8859060402684564

In [98]:
from sklearn.model_selection import GridSearchCV
parameters = {'vect__ngram_range': [(1, 1), (1, 2)],
              'clf__alpha': (1e-2, 1e-3)}

In [99]:
# from sklearn.model_selection import cross_val_score
# accuracies = cross_val_score(estimator = XGC, X =xt, y = yt, cv = 10)
# print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
# print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))